<a href="https://colab.research.google.com/github/rukmals/crowd-monitoring-system-model-development/blob/main/GT_Generation_SH_B_and_test_SH_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

import os


import os
from tensorflow.keras.preprocessing import image
import numpy as np
import scipy
from scipy.io import loadmat
import glob
import h5py
import time
from sklearn.externals.joblib import Parallel, delayed
from torchvision import datasets, transforms


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
import random
import os
from PIL import Image,ImageFilter,ImageDraw
import numpy as np
import h5py
from PIL import ImageStat
import glob
from sklearn.model_selection import train_test_split
import json

import sys
import warnings
# import from library
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import argparse
import json
import cv2
import time
from torchvision import models

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def gaussian_filter_density(gt):
    print(gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density
    #print(gt_count)
    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    leafsize = 2048
    # build kdtree
    pts_copy = pts.copy()
    tree = scipy.spatial.KDTree(pts_copy, leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)
    print('generate density...')
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1], pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1] + distances[i][2] + distances[i][3]) * 0.1
        else:
            sigma = np.average(np.array(gt.shape)) / 2. / 2.  # case: 1 point
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    print('done.')
    return density

In [ ]:
def generate_density_map(img_path):
    img_path = "/content/drive/MyDrive/Model_Test_Data/PartA/test_data/images/"+ img_path
    #print(img_path)
    mat_path = img_path.replace('.jpg', '.mat').replace('images', 'ground_truth').replace('IMG_', 'GT_IMG_')
    #print(mat_path)
    mat = scipy.io.loadmat(mat_path)
    imgfile = image.load_img(img_path)
    img = image.img_to_array(imgfile)
    k = np.zeros((img.shape[0], img.shape[1]))
    gt = mat["image_info"][0, 0][0, 0][0]
    for i in range(0, len(gt)):
        if int(gt[i][1]) < img.shape[0] and int(gt[i][0]) < img.shape[1]:
            k[int(gt[i][1]), int(gt[i][0])] = 1
    k = gaussian_filter_density(k)
    output_path = img_path.replace('.jpg', '.h5').replace('images','ground-truth-h5')
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)
    print("output", output_path)
    with h5py.File(output_path, 'w') as hf:
        hf['density'] = k
    return img_path

In [ ]:
test_list = os.listdir("/content/drive/MyDrive/Model_Test_Data/PartA/test_data/images")
print(test_list)

['IMG_110.jpg', 'IMG_108.jpg', 'IMG_10.jpg', 'IMG_102.jpg', 'IMG_105.jpg', 'IMG_103.jpg', 'IMG_100.jpg', 'IMG_101.jpg', 'IMG_11.jpg', 'IMG_104.jpg', 'IMG_1.jpg', 'IMG_107.jpg', 'IMG_106.jpg', 'IMG_109.jpg', 'IMG_116.jpg', 'IMG_126.jpg', 'IMG_112.jpg', 'IMG_113.jpg', 'IMG_120.jpg', 'IMG_127.jpg', 'IMG_119.jpg', 'IMG_117.jpg', 'IMG_114.jpg', 'IMG_12.jpg', 'IMG_123.jpg', 'IMG_124.jpg', 'IMG_115.jpg', 'IMG_118.jpg', 'IMG_111.jpg', 'IMG_125.jpg', 'IMG_121.jpg', 'IMG_128.jpg', 'IMG_122.jpg', 'IMG_14.jpg', 'IMG_151.jpg', 'IMG_153.jpg', 'IMG_144.jpg', 'IMG_147.jpg', 'IMG_130.jpg', 'IMG_129.jpg', 'IMG_142.jpg', 'IMG_138.jpg', 'IMG_131.jpg', 'IMG_15.jpg', 'IMG_135.jpg', 'IMG_148.jpg', 'IMG_134.jpg', 'IMG_140.jpg', 'IMG_150.jpg', 'IMG_152.jpg', 'IMG_136.jpg', 'IMG_133.jpg', 'IMG_132.jpg', 'IMG_13.jpg', 'IMG_141.jpg', 'IMG_137.jpg', 'IMG_149.jpg', 'IMG_143.jpg', 'IMG_145.jpg', 'IMG_139.jpg', 'IMG_146.jpg', 'IMG_172.jpg', 'IMG_157.jpg', 'IMG_169.jpg', 'IMG_167.jpg', 'IMG_16.jpg', 'IMG_171.jpg', 'IM

In [ ]:
print(len(test_list))

364


In [ ]:
generate_density_map('IMG_1.jpg')

/content/drive/MyDrive/Model_Test_Data/PartA/test_data/images/IMG_1.jpg
/content/drive/MyDrive/Model_Test_Data/PartA/test_data/ground_truth/GT_IMG_1.mat
(704, 1024)
generate density...
done.
output /content/drive/MyDrive/Model_Test_Data/PartA/test_data/ground-truth-h5/IMG_1.h5


'/content/drive/MyDrive/Model_Test_Data/PartA/test_data/images/IMG_1.jpg'

In [ ]:
for path in test_list:
  generate_density_map(path)

In [4]:
gt_test_list = os.listdir("/content/drive/MyDrive/Model_Test_Data/PartA/test_data/ground-truth-h5")
print(len(gt_test_list))

182


In [5]:
import os
import random
import torch
import numpy as np
from torch.utils.data import Dataset
from PIL import Image

def load_data(img_path,train = True):
    gt_path = img_path.replace('.jpg','.h5').replace('images','ground-truth-h5')
    img = Image.open(img_path).convert('RGB')
    gt_file = h5py.File(gt_path, 'r')
    target = np.asarray(gt_file['density'])
    #target = cv2.resize(target,(int(target.shape[1]/2.25), int(target.shape[0]/2.25)),interpolation = cv2.INTER_CUBIC)
    return img,target

In [6]:
class ListDataset(Dataset):
    def __init__(self, root, shape=None, shuffle=True, transform=None,  train=False, batch_size=1, num_workers=4):
        """
        if you have different image size, then batch_size must be 1
        :param root:
        :param shape:
        :param shuffle:
        :param transform:
        :param train:
        :param seen:
        :param batch_size:
        :param num_workers:
        """
        #if train:
            #root = root *4
        if shuffle:
            random.shuffle(root)
        
        self.nSamples = len(root)
        self.lines = root
        self.transform = transform
        self.train = train
        self.shape = shape
        self.batch_size = batch_size
        self.num_workers = num_workers
        
    def __len__(self):
        return self.nSamples

    def __getitem__(self, index):
        assert index <= len(self), 'index range error' 
        
        img_path = self.lines[index]
        
        img,target = load_data(img_path,self.train)
        
        #img = 255.0 * F.to_tensor(img)
        
        #img[0,:,:]=img[0,:,:]-92.8207477031
        #img[1,:,:]=img[1,:,:]-95.2757037428
        #img[2,:,:]=img[2,:,:]-104.877445883
        if self.transform is not None:
            img = self.transform(img)
        return img,target
  

In [15]:
img_test_list = os.listdir("/content/drive/MyDrive/Model_Test_Data/PartB/test_data/images")
img_list = []
for img in img_test_list:
  if '.jpg' in img:
    img_list.append(img)


In [24]:
img_list.remove("IMG_239 (1).jpg")

In [25]:
len(img_list)

316

In [26]:
sh_test_list = []
for img_ in img_list:
  img_ = "/content/drive/MyDrive/Model_Test_Data/PartB/test_data/images/"+img_
  sh_test_list.append(img_)

In [27]:
len(sh_test_list)

316

In [18]:
test_loader = torch.utils.data.DataLoader(ListDataset(sh_test_list,
                                  shuffle=False,
                                  transform=transforms.Compose([transforms.ToTensor(), transforms.Resize(480),transforms.Normalize(mean=[0.302234709263, 0.291243076324, 0.269087553024],
                                                                                                                  std=[0.227743327618, 0.211051672697, 0.184846073389]),
                                                                  ]), train=False,batch_size=2,num_workers=2),
              batch_size=2)

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class Conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, \
                stride=1, NL='relu', same_padding=False, bn=False, dilation=1):
        super(Conv2d, self).__init__()
        padding = int((kernel_size - 1) // 2) if same_padding else 0
        self.conv = []
        if dilation==1:
            self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=padding, dilation=dilation)
        else:
            self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=dilation, dilation=dilation)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001, momentum=0, affine=True) if bn else nn.Identity()
        if NL == 'relu' :
            self.relu = nn.ReLU(inplace=True)
        elif NL == 'prelu':
            self.relu = nn.PReLU()
        else:
            self.relu = None

    def forward(self, x):
      x = self.conv(x)
      if self.bn is not None:
          x = self.bn(x)
      if self.relu is not None:
          x = self.relu(x)   
      return x
  
# the module definition for the multi-branch in the density head
class MultiBranchModule(nn.Module):
    def __init__(self, in_channels, sync=False):
        super(MultiBranchModule, self).__init__()
        self.branch_column1_1 = BasicConv2d(in_channels, in_channels//2, kernel_size=1, sync=sync)
        self.branch_column1_2 = BasicConv2d(in_channels//2, in_channels, kernel_size=1, sync=sync)

        self.branch_column2_1 = BasicConv2d(in_channels, in_channels//2, kernel_size=1, sync=sync)
        self.branch_column2_2 = BasicConv2d(in_channels // 2, in_channels, kernel_size=(3, 3), padding=(1, 1), sync=sync)

        self.branch_column3_1 = BasicConv2d(in_channels, in_channels//2, kernel_size=1, sync=sync)
        self.branch_column3_2 = BasicConv2d(in_channels // 2, in_channels, kernel_size=5, padding=2, sync=sync)

    def forward(self, x):
        branch_1 = self.branch_column1_1(x)
        branch_1 = self.branch_column1_2(branch_1)

        branch_2 = self.branch_column2_1(x)
        branch_2 = self.branch_column2_2(branch_2)

        branch_3 = self.branch_column3_1(x)
        branch_3 = self.branch_column3_2(branch_3)

        outputs = [branch_1, branch_2, branch_3, x]
        return torch.cat(outputs, 1)

# the module definition for the basic conv module
class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, sync=False, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        if sync:
            # for sync bn
            print('use sync inception')
            self.bn = nn.SyncBatchNorm(out_channels, eps=0.001)
        else:
            self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

In [20]:
class TestNet(nn.Module):
    def __init__(self, pretrained=True):
        super(TestNet, self).__init__()
        
        vgg = models.vgg16_bn(pretrained=pretrained)
        
        self.backend_feat  = [256,128,64]


        # Front End Development VGG - 16 
        features = list(vgg.features.children())
        # get each stage of the VGG - 16
        self.features1 = nn.Sequential(*features[0:6])
        self.features2 = nn.Sequential(*features[6:13])
        self.features3 = nn.Sequential(*features[13:23])
        self.features4 = nn.Sequential(*features[23:33])
        self.features5 = nn.Sequential(*features[33:43])

        # Front End Development P1 to P5 
        self.p5 = nn.Sequential(
            Conv2d(512, 1024, 3, same_padding=True, NL='relu'),
            Conv2d(1024, 512, 3, same_padding=True, NL='relu'),
        )

        self.p4 = nn.Sequential(
            Conv2d(1024, 512, 3, same_padding=True, NL='relu'),
            Conv2d(512, 256, 3, same_padding=True, NL='relu'),
        )

        self.p3 = nn.Sequential(
            Conv2d(512 , 256, 3, same_padding=True, NL='relu'),
            Conv2d(256, 128, 3, same_padding=True, NL='relu'),
        )

        self.p2 = nn.Sequential(
            Conv2d(256, 128, 3, same_padding=True, NL='relu'),
            Conv2d(128, 64, 3, same_padding=True, NL='relu'),
        )

        self.p1 = nn.Sequential(
            Conv2d(128, 64, 3, same_padding=True, NL='relu'),
            Conv2d(64, 64, 3, same_padding=True, NL='relu'),
        ) 

        # Multi-Branch moules
        self.multi_branch5 = nn.Sequential(
            MultiBranchModule(512),
            Conv2d(2048, 1, 1, same_padding=True)
        )

        self.multi_branch4 = nn.Sequential(
            MultiBranchModule(256),
            Conv2d(1024, 1, 1, same_padding=True)
        )

        self.multi_branch3 = nn.Sequential(
            MultiBranchModule(128),
            Conv2d(512, 1, 1, same_padding=True)
        )

        self.multi_branch2 = nn.Sequential(
            MultiBranchModule(64),
            Conv2d(256, 1, 1, same_padding=True)
        )

        self.multi_branch1 = nn.Sequential(
            MultiBranchModule(64),
            Conv2d(256, 1, 1, same_padding=True)
        )

        self.backend = make_layers(self.backend_feat,in_channels = 5,dilation = True)

        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)

    
    
    def forward(self, x):
        size = x.size()
        x1 = self.features1(x)
        x2 = self.features2(x1)
        x3 = self.features3(x2)
        x4 = self.features4(x3)
        x5 = self.features5(x4)

        # Front End Development P1 to P5 
        x = self.p5(x5)
        x5_out = x
        x = F.upsample_bilinear(x, size=x4.size()[2:])

        x = torch.cat([x4, x], 1)
        x = self.p4(x)
        x4_out = x
        x = F.upsample_bilinear(x, size=x3.size()[2:])

        x = torch.cat([x3, x], 1)
        x = self.p3(x)
        x3_out = x
        x = F.upsample_bilinear(x, size=x2.size()[2:])

        x = torch.cat([x2, x], 1)
        x = self.p2(x)
        x2_out = x
        x = F.upsample_bilinear(x, size=x1.size()[2:])

        x = torch.cat([x1, x], 1)
        x = self.p1(x)
        x1_out = x


        # multi-branch predictions
        x5_density = self.multi_branch5(x5_out)
        x4_density = self.multi_branch4(x4_out)
        x3_density = self.multi_branch3(x3_out)
        x2_density = self.multi_branch2(x2_out)
        x1_density = self.multi_branch1(x1_out)

        # upsample the multi-branch predictions to be the same with the input size
        x5_density = F.upsample_nearest(x5_density, size=x1.size()[2:])
        x4_density = F.upsample_nearest(x4_density, size=x1.size()[2:])
        x3_density = F.upsample_nearest(x3_density, size=x1.size()[2:])
        x2_density = F.upsample_nearest(x2_density, size=x1.size()[2:])
        x1_density = F.upsample_nearest(x1_density, size=x1.size()[2:])


        density_map = torch.cat([x5_density, x4_density, x3_density, x2_density, x1_density], 1)


        x_out = self.backend(density_map)
        density_map_out = self.output_layer(x_out)
        return density_map_out
        #return density_map
                
                
def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    layers = []
    dilation_rates = [2,3,5]
    #for v in cfg:
    for v in range(len(cfg)):
        if cfg[v] == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, cfg[v], kernel_size=3, padding=dilation_rates[v],dilation = dilation_rates[v])
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(cfg[v]), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = cfg[v]
    return nn.Sequential(*layers)

In [21]:
model_path = '/content/drive/MyDrive/GCC_CSV_DataSet/model/TestNetGCC_whole_data_withbackend.pth'

model = TestNet().cuda()
# load the trained model
model.load_state_dict(torch.load(model_path))
print('successfully load model from', model_path)

successfully load model from /content/drive/MyDrive/GCC_CSV_DataSet/model/TestNetGCC_whole_data_withbackend.pth


In [22]:
pred_transform=transforms.Compose([transforms.ToTensor(), transforms.Resize(480), transforms.Normalize(mean=[0.302234709263, 0.291243076324, 0.269087553024],
                                                                                                                std=[0.227743327618, 0.211051672697, 0.184846073389]),])



In [ ]:
sh_im = Image.open('/content/drive/MyDrive/Model_Test_Data/PartA/test_data/images/IMG_1.jpg')
sh_pred = model(pred_transform(sh_im)[None].cuda())

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3825: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3770: UserWarning: nn.functional.upsample_nearest is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_nearest is deprecated. Use nn.functional.interpolate instead.")


In [28]:
mae = 0
for im_path in sh_test_list:
  image = Image.open(im_path).convert('RGB')
  #print(image.shape)
  output = model(pred_transform(image)[None].cuda())
  gt_path = im_path.replace('.jpg','.h5').replace('images','ground-truth-h5')
  gt_file = h5py.File(gt_path, 'r')
  target = np.asarray(gt_file['density'])
  print("ground trueth count", np.sum(target))
  output = output.cpu()
  output = output.detach().numpy()
  print("Model Predicted Count",np.sum(output)/100)
  
  mae += abs(np.sum(target) -np.sum(output)/100)

mae = mae / len(sh_test_list)
print(' * TESTING MAE {mae:.3f} '.format(mae=mae))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3825: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3770: UserWarning: nn.functional.upsample_nearest is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_nearest is deprecated. Use nn.functional.interpolate instead.")


ground trueth count 172.99678
Model Predicted Count 182.751171875
ground trueth count 21.089602
Model Predicted Count 18.167713623046875
ground trueth count 33.12424
Model Predicted Count 30.04722412109375
ground trueth count 64.97026
Model Predicted Count 93.914111328125
ground trueth count 156.47859
Model Predicted Count 143.89033203125
ground trueth count 53.26684
Model Predicted Count 72.0537548828125
ground trueth count 41.2734
Model Predicted Count 44.1030615234375
ground trueth count 470.27805
Model Predicted Count 635.7169921875
ground trueth count 160.92084
Model Predicted Count 188.71787109375
ground trueth count 216.71764
Model Predicted Count 297.42931640625
ground trueth count 103.04705
Model Predicted Count 110.851240234375
ground trueth count 156.65094
Model Predicted Count 166.39474609375
ground trueth count 132.69264
Model Predicted Count 135.035751953125
ground trueth count 17.747625
Model Predicted Count 24.57567138671875
ground trueth count 302.12555
Model Predicted

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=c1c52ebd494bb4a41344a15303a816d9d47a40102e18d143069265e71156ee44
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil


In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


ModuleNotFoundError: ignored

In [ ]:
import torch, gc
#with torch.no_grad():
gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 14           |        cudaMalloc retries: 14        |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    4672 MB |    9685 MB |   78618 MB |   73946 MB |\n|       from large pool |    4664 MB |    9677 MB |   78591 MB |   73926 MB |\n|       from small pool |       7 MB |      13 MB |      26 MB |      19 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |    4672 MB |    9685 MB |   78618 MB |   73946 MB |\n|       from large pool |    4664 MB |    9677 MB |